## City of Toronto Bicycle Network Data

## Setup Notebook

In [5]:
# Import 3rd party libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pylab as plt
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from IPython.display import display
import geopandas as gpd
from shapely.geometry import Point

# Configure Notebook
%matplotlib inline
plt.style.use('fivethirtyeight')
sns.set_context("notebook")
import warnings
warnings.filterwarnings('ignore')

## Import GeoJson Data

In [7]:
# Import dataset as a GeoDataFrame
bicycle_data = gpd.read_file('cycling-network - 4326.geojson')

bicycle_data.tail()


,_id,OBJECTID,SEGMENT_ID,INSTALLED,UPGRADED,PRE_AMALGAMATION,STREET_NAME,FROM_STREET,TO_STREET,ROADCLASS,...,INFRA_HIGHORDER,SEPA_HIGHORDER,SEPB_HIGHORDER,ORIG_HIGHORDER,BYLAWED,EDITOR,LAST_EDIT_DATE,UPGRADE_DESCRIPTION,CONVERTED,geometry
1459,1460,1460,1460,2023,0,,Dundas St W,Beamish Dr,Beamish Dr,,...,Cycle Track,,,,,,2024-06-11 16:40:31,,,"MULTILINESTRING ((-79.53625 43.64, -79.53623 4..."
1460,1461,1461,1461,2023,0,,Bloor St W,Beamish Dr,Beamish Dr,,...,Cycle Track,,,,,,2024-06-11 16:40:31,,,"MULTILINESTRING ((-79.53746 43.64136, -79.5372..."
1461,1462,1462,1462,2023,0,,Kipling Ave,St Albans Rd,St Albans Rd,,...,Cycle Track,,,,,,2024-06-11 16:40:31,,,"MULTILINESTRING ((-79.53433 43.63922, -79.5342..."
1462,1463,1463,1463,2023,0,,Dundas St W,Jopling Ave S,Jopling Ave S,,...,Cycle Track,,,,,,2024-06-11 16:40:31,,,"MULTILINESTRING ((-79.53755 43.63906, -79.5376..."
1463,1464,1464,1464,2024,0,,University Ave,Wellington St W,[150m S] Wellington St W,,...,Cycle Track,,,,,,2024-06-11 16:40:31,,,"MULTILINESTRING ((-79.3841 43.64638, -79.38386..."


## Data Analysis

In [9]:
# Check the number of columns and rows
bicycle_data.shape

(1464, 29)

In [10]:
# Check the columns in DataFrame
bicycle_data.columns

Index(['_id', 'OBJECTID', 'SEGMENT_ID', 'INSTALLED', 'UPGRADED',
       'PRE_AMALGAMATION', 'STREET_NAME', 'FROM_STREET', 'TO_STREET',
       'ROADCLASS', 'CNPCLASS', 'SURFACE', 'OWNER', 'DIR_LOWORDER',
       'INFRA_LOWORDER', 'SEPA_LOWORDER', 'SEPB_LOWORDER',
       'ORIG_LOWORDER_INFRA', 'DIR_HIGHORDER', 'INFRA_HIGHORDER',
       'SEPA_HIGHORDER', 'SEPB_HIGHORDER', 'ORIG_HIGHORDER', 'BYLAWED',
       'EDITOR', 'LAST_EDIT_DATE', 'UPGRADE_DESCRIPTION', 'CONVERTED',
       'geometry'],
      dtype='object')

- _id: A unique identifier for each record. Typically used to uniquely reference each row in the dataset.
- OBJECTID: Another unique identifier, often used in geographic or GIS datasets to represent the object in a spatial database. This ID helps to track features in the system.
- SEGMENT_ID:An identifier for a specific segment of road, street, or infrastructure. It likely refers to a portion of a larger infrastructure asset, useful for analysis of road conditions or maintenance.
- INSTALLED:The date or year when the infrastructure (e.g., road, street feature, or installation) was installed. This can be important for asset management and aging infrastructure analysis.
- UPGRADED: Indicates whether the infrastructure has been upgraded, often with a specific date or year of the upgrade. This field is useful for understanding the timeline of infrastructure improvements.
- PRE_AMALGAMATION: A flag or value indicating whether the feature existed before a certain amalgamation event (e.g., before the merging of municipalities, districts, or other administrative boundaries).
- STREET_NAME: The name of the street or road that the feature corresponds to. This is crucial for identifying and mapping locations.
- FROM_STREET: The starting street or intersection for a particular road segment. It helps in defining the segment's geographic location and orientation.
- TO_STREET: The ending street or intersection for the road segment. This, together with the FROM_STREET, helps define the entire segment’s extent.
- ROADCLASS: The classification of the road based on factors like its size, importance, and traffic capacity (e.g., arterial, residential, highway). This can help in traffic planning and infrastructure management.
- CNPCLASS: Likely refers to a specific classification type relevant to the road or asset (could be related to a "CNP" code system, which might relate to "Class of Network Plan" or something similar, depending on local regulations or standards).
- SURFACE:The type of surface material for the road (e.g., asphalt, concrete, gravel). This helps in maintenance planning and determining the durability of the road.
- OWNER: The entity or agency that owns the infrastructure asset. This could refer to municipal, provincial, or private ownership, and is important for responsibility and asset management.
- DIR_LOWORDER: A directional classification, likely referring to the "low order" roads or lanes within a given system (possibly lower-traffic or smaller roads in a hierarchy).
- INFRA_LOWORDER: Indicates infrastructure components that are part of the lower-order (or minor) part of the road system. These could be related to smaller roads, curbs, or other features.
- SEPA_LOWORDER:May refer to a low-order separation feature such as sidewalks or barriers, depending on local definitions. Could be part of a classifying system for features in road infrastructure.
- SEPB_LOWORDER: Similar to SEPA_LOWORDER, this likely refers to another kind of separation feature (e.g., barriers or buffers) associated with lower-order roads.
- ORIG_LOWORDER_INFRA: Refers to the original infrastructure classification for low-order roads or features. This might track historical data or previous classifications before any changes or upgrades were made.
- DIR_HIGHORDER: A directional classification for higher-order roads or lanes (e.g., major highways, arterial roads). These roads typically handle more traffic and are crucial for transportation planning.
- INFRA_HIGHORDER: Refers to infrastructure components associated with high-order roads, such as major roadways, highways, and interchanges.
- SEPA_HIGHORDER: A separation feature for higher-order roads, which could include things like dividers, buffers, or barriers designed for major roadways.
- SEPB_HIGHORDER: Similar to SEPA_HIGHORDER, but might refer to a different type or class of separation feature (e.g., barriers or safety features) for high-order roads.
- ORIG_HIGHORDER: The original classification or state of the high-order infrastructure, before any changes, upgrades, or reclassification.
- BYLAWED:
This field could indicate whether the infrastructure feature is subject to specific bylaws or regulations (e.g., zoning or building codes). It may signify restrictions or requirements tied to the feature.
- EDITOR:The name or ID of the person or entity who last edited or modified the record. This is important for tracking changes and maintaining the integrity of the data.
- LAST_EDIT_DATE:The date when the feature was last modified or updated. It helps track the freshness of the data and when changes were made.
- UPGRADE_DESCRIPTION:A description of any upgrades or improvements that have been made to the infrastructure. This could include road widening, resurfacing, or additions of new features.
- CONVERTED:A flag or value indicating whether the infrastructure feature has been converted to another type, such as from one road class to another or from one material to another.
- geometry:The geometric representation of the infrastructure feature, typically stored as Shapely geometry objects. This column contains spatial data (points, lines, or polygons) that represent the physical location and shape of the asset.

In [42]:
# Drop columns
bicycle_data = bicycle_data.drop(columns=[
    'OBJECTID',
    'SEGMENT_ID',                  
    'UPGRADED',      
    'PRE_AMALGAMATION',      
    'FROM_STREET',              
    'TO_STREET',               
    'CNPCLASS',               
    'SURFACE',               
    'OWNER',              
    'INFRA_LOWORDER',
    'DIR_LOWORDER',             
    'SEPA_LOWORDER',             
    'SEPB_LOWORDER',        
    'ORIG_LOWORDER_INFRA',          
    'DIR_HIGHORDER',             
    'INFRA_HIGHORDER', 
    'SEPA_HIGHORDER',
    'SEPB_HIGHORDER',
    'ORIG_HIGHORDER',
    'BYLAWED',
    'EDITOR',
    'LAST_EDIT_DATE',
    'UPGRADE_DESCRIPTION',
    'CONVERTED',
    'ROADCLASS'
], errors='ignore')

# Check if columns are removed and remaining columns have been renamed
bicycle_data.columns

Index(['_id', 'INSTALLED', 'STREET_NAME', 'geometry'], dtype='object')

In [44]:
bicycle_data.head()

,_id,INSTALLED,STREET_NAME,geometry
0,1,2001,Kilbarry Rd,"MULTILINESTRING ((-79.40351 43.69526, -79.4030..."
1,2,2001,Martin Goodman / Waterfront Trl,"MULTILINESTRING ((-79.40364 43.63499, -79.4035..."
2,3,2001,Gatineau Hydro Corridor Trl,"MULTILINESTRING ((-79.27522 43.74158, -79.2751..."
3,4,2001,G Ross Lord Park Trl,"MULTILINESTRING ((-79.46772 43.77208, -79.4675..."
4,5,2001,Etobicoke Creek Trl,"MULTILINESTRING ((-79.54812 43.59027, -79.5480..."


In [46]:
# Check data types per column
print(bicycle_data.dtypes)

_id               int32
INSTALLED         int32
STREET_NAME      object
geometry       geometry
dtype: object


In [48]:
# Check numerical statistics for each column
bicycle_data.describe()

,_id,INSTALLED
count,1464.00000,1464.000000
mean,732.50000,1987.971311
std,422.76471,202.477544
min,1.00000,0.000000
25%,366.75000,2001.000000
50%,732.50000,2006.000000
75%,1098.25000,2015.000000
max,1464.00000,2024.000000


In [50]:
# Check for missing values
print(bicycle_data.isnull().sum())

# Drop rows with any NaN values
bicycle_data_clean = bicycle_data.dropna()

# Verify the result by checking the number of missing values after dropping NaNs
print(bicycle_data_clean.isnull().sum())


_id            0
INSTALLED      0
STREET_NAME    0
geometry       0
dtype: int64
_id            0
INSTALLED      0
STREET_NAME    0
geometry       0
dtype: int64


In [52]:
bicycle_data_clean.shape

(1464, 4)

In [54]:
# View DataFrame
bicycle_data_clean.head(100)

,_id,INSTALLED,STREET_NAME,geometry
0,1,2001,Kilbarry Rd,"MULTILINESTRING ((-79.40351 43.69526, -79.4030..."
1,2,2001,Martin Goodman / Waterfront Trl,"MULTILINESTRING ((-79.40364 43.63499, -79.4035..."
2,3,2001,Gatineau Hydro Corridor Trl,"MULTILINESTRING ((-79.27522 43.74158, -79.2751..."
3,4,2001,G Ross Lord Park Trl,"MULTILINESTRING ((-79.46772 43.77208, -79.4675..."
4,5,2001,Etobicoke Creek Trl,"MULTILINESTRING ((-79.54812 43.59027, -79.5480..."
...,...,...,...,...
95,96,2001,Lower Don River Trl,"MULTILINESTRING ((-79.36203 43.68813, -79.3620..."
96,97,2001,Colborne Lodge Dr,"MULTILINESTRING ((-79.46208 43.64182, -79.4620..."
97,98,2001,Eglinton Ave W Trail,"MULTILINESTRING ((-79.58431 43.66947, -79.5842..."
98,99,2001,Eglinton Ave W Trail,"MULTILINESTRING ((-79.58431 43.66947, -79.5842..."


In [56]:
# Calculate the percentages of each unique 'INSTALLED' value
installed_counts = bicycle_data_clean['INSTALLED'].value_counts(normalize=True) * 100
installed_counts_df = installed_counts.reset_index()
installed_counts_df.columns = ['INSTALLED', 'PERCENTAGE']

# Sort the DataFrame by the 'INSTALLED' column in ascending order
installed_counts_df_sorted = installed_counts_df.sort_values(by='INSTALLED', ascending=True)

# Reset the index for better readability
installed_counts_df_sorted.reset_index(drop=True, inplace=True)

# Display the sorted DataFrame
print(installed_counts_df_sorted)

    INSTALLED  PERCENTAGE
0           0    1.024590
1        2001   36.816940
2        2002    0.819672
3        2003    1.024590
4        2004    1.366120
5        2005    4.439891
6        2006    5.122951
7        2007    6.079235
8        2008    1.639344
9        2009    2.254098
10       2010    0.751366
11       2011    3.551913
12       2012    4.030055
13       2013    1.707650
14       2014    2.732240
15       2015    4.098361
16       2016    1.775956
17       2017    1.912568
18       2018    3.073770
19       2019    1.502732
20       2020    3.073770
21       2021    2.800546
22       2022    4.986339
23       2023    3.210383
24       2024    0.204918


In [58]:
# Calculate the percentage of data with 'INSTALLED' values before 2014
percent_before_2014 = installed_counts_df_sorted.loc[installed_counts_df_sorted['INSTALLED'] < 2014, 'PERCENTAGE'].sum()

# Print the result
print(f"Percentage of data before 2014: {percent_before_2014}%")

Percentage of data before 2014: 70.62841530054645%


In [62]:
# Path to the GeoJSON file
geojson_path = "../Clean Data/wards_data.geojson"

# Load the shapefile
wards_data = gpd.read_file(geojson_path)

# View geoDataFrame
wards_data.head()

,AREA_ID,AREA_TYPE,AREA_S_CD,AREA_NAME,X,Y,LONGITUDE,LATITUDE,geometry
0,2551040,WD18,16,Don Valley East,318237.290,4844000.0,-79.33298,43.739716,"POLYGON ((-79.31335 43.71699, -79.3195 43.7156..."
1,2551044,WD18,03,Etobicoke-Lakeshore,303099.474,4831000.0,-79.52087,43.621646,"POLYGON ((-79.49777 43.65198, -79.49725 43.651..."
2,2551048,WD18,15,Don Valley West,314825.876,4843000.0,-79.37536,43.728396,"POLYGON ((-79.35232 43.71573, -79.35209 43.715..."
3,2551052,WD18,23,Scarborough North,324522.149,4852000.0,-79.25467,43.809672,"POLYGON ((-79.22591 43.8396, -79.22556 43.8394..."
4,2551056,WD18,11,University-Rosedale,313306.543,4837000.0,-79.39432,43.671139,"POLYGON ((-79.39004 43.6905, -79.39004 43.6905..."


In [66]:
# Convert CRS to EPSG:32617
bicycle_data_clean = bicycle_data_clean.to_crs("EPSG:32617")  
wards_data = wards_data.to_crs("EPSG:32617")

In [74]:
# Perform the spatial join
connected_data = gpd.sjoin(bicycle_data_clean, wards_data[['AREA_NAME', 'AREA_S_CD', 'geometry']], how="inner", predicate="intersects")


# Save or display the resulting GeoDataFrame
connected_data.head()

,_id,INSTALLED,STREET_NAME,geometry,index_right,AREA_NAME,AREA_S_CD
0,1,2001,Kilbarry Rd,"MULTILINESTRING ((628651.515 4839265.273, 6286...",13,Toronto-St. Paul's,12
1,2,2001,Martin Goodman / Waterfront Trl,"MULTILINESTRING ((628769.688 4832571.759, 6287...",5,Spadina-Fort York,10
2,3,2001,Gatineau Hydro Corridor Trl,"MULTILINESTRING ((638882.149 4844617.068, 6388...",15,Scarborough Centre,21
3,4,2001,G Ross Lord Park Trl,"MULTILINESTRING ((623319.222 4847699.258, 6233...",21,York Centre,06
4,5,2001,Etobicoke Creek Trl,"MULTILINESTRING ((617202.25 4827390.434, 61720...",1,Etobicoke-Lakeshore,03


In [76]:
result_renamed = connected_data.rename(columns={'AREA_S_CD': 'WARD NUMBER', 'AREA_NAME': 'WARD NAME'})
result_final = result_renamed.drop(columns=['index_right'])

# Display the result
result_final.head()

,_id,INSTALLED,STREET_NAME,geometry,WARD NAME,WARD NUMBER
0,1,2001,Kilbarry Rd,"MULTILINESTRING ((628651.515 4839265.273, 6286...",Toronto-St. Paul's,12
1,2,2001,Martin Goodman / Waterfront Trl,"MULTILINESTRING ((628769.688 4832571.759, 6287...",Spadina-Fort York,10
2,3,2001,Gatineau Hydro Corridor Trl,"MULTILINESTRING ((638882.149 4844617.068, 6388...",Scarborough Centre,21
3,4,2001,G Ross Lord Park Trl,"MULTILINESTRING ((623319.222 4847699.258, 6233...",York Centre,06
4,5,2001,Etobicoke Creek Trl,"MULTILINESTRING ((617202.25 4827390.434, 61720...",Etobicoke-Lakeshore,03


In [78]:
output_path = "../Clean Data/bicycle_data.geojson"
result_final.to_file(output_path, driver="GeoJSON")